In [ ]:
import sys
sys.path.append(r'Q:\sachuriga\Sachuriga_Python\quattrocolo-nwb4fp\src/')
sys.path.append(r'Q:\sachuriga\Sachuriga_Python\cnn-ripple-main/src/')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nwb4fp.analyses.data import get_pkl_files, group_channels_by_group, find_middle_channel_per_group, map_middle_channels_to_input,get_nearest_8_by_position, load_speed_fromNWB, pos2speed, find_run_indices
from nwb4fp.preprocess.down_sample_lfp import down_sample_lfp_test
import pynapple as nap
from cnn.load_data import z_score_normalization, downsample_data
from cnn.utils import smooth2ripple
from cnn.load_data import generate_overlapping_windows


file_path = r"Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp/ASSY-236-F.prb"
base_folder = r"S:/Sachuriga/Ephys_Recording/CR_CA1"
base_nwb_fodler  =r"S:\Sachuriga/nwb/test4neo"
# Read the file and parse the dictionary
local_vars = {'np': np}
with open(file_path, 'r') as file:
    exec(file.read(), local_vars)  # Execute the file content with NumPy in scope

channel_groups = local_vars.get('channel_groups')
if channel_groups is None:
    raise ValueError(f"'channel_groups' not found in {file_path}")

# Assuming channel_groups is loaded from Step 1
data = []
for group_id, group_data in channel_groups.items():
    channels = group_data['channels']
    geometry = group_data['geometry']
    for channel in channels:
        x, y = geometry[channel]
        data.append({
            'group_id': group_id,
            'channel_id': channel,
            'x': x,
            'y': y
        })
probe_df = pd.DataFrame(data)
base_tsne_folder = r"S:\Sachuriga\file_with_table"
pkl_files = get_pkl_files(rf"{base_tsne_folder}")
base_tsne_ripplemax_folder = r"S:\Sachuriga\file_with_table\ripple_ch"
all_dataframes = []

overall_error_log=[]
#for file in pkl_files:
file=r"65622_2024-04-30_14-59-30_units_table_withDLC.pkl"
try:
    # Read and process the pickle file
    print(fr"working on this folder {file}")
    df_file = pd.read_pickle(fr"{base_tsne_folder}/{file}")

    #df_file = pd.read_pickle(fr"S:/Sachuriga/filre_with_table/adjust_y_with _meanVAlue/63385_2024-07-22_B_units_table_withDLC.pkl")
    unit_table = pd.DataFrame(df_file)
    # pyramidal_df = unit_table[unit_table['cell_type']=="pyramidal"]

    # file_path_nwb = pyramidal_df['session_id'].iloc[0]
    # channel_list = pyramidal_df['ch'].values

    file_path_nwb = unit_table['session_id'].iloc[0]
    channel_list = unit_table['ch'].values
    
    # Channel grouping and calculations
    grouped_channels = group_channels_by_group(channel_list)
    middle_channels = find_middle_channel_per_group(grouped_channels)
    output_list = map_middle_channels_to_input(channel_list, middle_channels)
    # output_list
    from nwb4fp.preprocess.down_sample_lfp import down_sample_lfp_test
    animal_id = unit_table['matlab_animal'].iloc[0]
    nwb_file = unit_table['session_id'].iloc[0]
    phy_folder = unit_table['session_id'].iloc[0].split(".nwb")[0]
    os_fodler = unit_table['session_id'].iloc[0].split("_phy_k_manual.nwb")[0]
    temp_folder = r"C:/temp_lfp"
    path = fr"{base_folder }/{animal_id}/{phy_folder}/"
    raw_path = fr"{base_folder}/{animal_id}/{os_fodler}"
    npdata = nap.load_file(fr"{base_nwb_fodler}/{nwb_file}")
    #down_sample_lfp_test(path,raw_path)
    pos_cord = load_speed_fromNWB(npdata['XY_mid_brain'])
    ## filter speed
    raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                                pos_cord[:,1], # x
                                pos_cord[:,2], # y
                                filter_speed=True, 
                                min_speed = 0.1)
    time_stemp = pos_cord[:,0]
    starts,stops = find_run_indices(smoothed_speed, threshold=0.05)
    run_ep = nap.IntervalSet(start=time_stemp[starts], end=time_stemp[stops], time_units='s')
    wake_ep = nap.IntervalSet(start=time_stemp[0], end=time_stemp[-1], time_units='s')
    rest_ep = wake_ep.set_diff(run_ep)

    ## find max
    try:
        lfp = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_zscore.npy")
        lfp_times = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_times.npy")
    except Exception as e:
        down_sample_lfp_test(path,raw_path)
        lfp = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_zscore.npy")
        lfp_times = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_times.npy")
    try:
        bad_channels = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/bad_channels.npy")
    except Exception as e:
        bad_channels = []
    result_dict={}
    ripple_channel={}
    ripple_channel_3std={}
    ripple_channel_4std={}
    for group, ch in middle_channels.items():
        temp = probe_df[probe_df['group_id']==group]
        temp_chs = temp.sort_values('y', ascending=False)
        chs = temp_chs['channel_id'].values
        # Example usage
        numbers = list(chs)
        target = np.int64(ch)
        bad_channels=list(bad_channels)
        result = get_nearest_8_by_position(numbers, target, bad_channels)
        
        print(fr"group is {group}, midlle channel is {ch}")
        print(fr"results is {result}")

        # Data preprocessing (unchanged)
        #temp = [31, 12, 23, 28, 15, 29, 7, 13]  # Electrode indices
        temp_lfps = []
        temp_lfps_times = []
        temp = result
        # Extract LFP signals for selected electrodes
        for ch in temp:
            temp_lfp = lfp[:, ch]
            eeg_examples = nap.Tsd(t=lfp_times, d=temp_lfp, fs=1250)
            signal = eeg_examples.restrict(rest_ep).values
            t = eeg_examples.restrict(rest_ep).index.values
            temp_lfps.append(list(signal))
            temp_lfps_times = list(t)

        # Convert to numpy array and normalize
        raw_lfp = np.array(temp_lfps).T
        data = z_score_normalization(raw_lfp)
        swr_events, max_electrode = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=2)
        swr_events1, max_electrode1 = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=3)
        swr_events2, max_electrode2 = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=4)
        ripple_channel[group] = temp[max_electrode]
        ripple_channel_3std[group] = temp[max_electrode1]
        ripple_channel_4std[group] = temp[max_electrode2]
    df_file['ripple_ch']=None
    df_file['ripple_ch_3std']=None
    df_file['ripple_ch_4std']=None
    df_file['shank_group']=None
    error_log=[]
    for i in range(len(df_file)):
        unit_group = probe_df[probe_df['channel_id']==df_file['ch'].iloc[i]]['group_id']
        try:
            df_file['ripple_ch'].iloc[i] = ripple_channel[unit_group.values[0]]
            df_file['ripple_ch_3std'].iloc[i] = ripple_channel_3std[unit_group.values[0]]
            df_file['ripple_ch_4std'].iloc[i] = ripple_channel_4std[unit_group.values[0]]
            df_file['shank_group'].iloc[i] = unit_group.values[0]
        except Exception as e:
            error_log.append(e)
            continue
    df_file.to_pickle(fr"{base_tsne_ripplemax_folder}/{file}")
except Exception as e:
    overall_error_log.append({file : e})


working on this folder 63385_2024-07-20_17-08-19_units_table_withDLC.pkl


C:\Users\sachur\AppData\Local\Temp\ipykernel_4180\1053226298.py:91: UserWarning: Some epochs have no duration
  run_ep = nap.IntervalSet(start=time_stemp[starts], end=time_stemp[stops], time_units='s')


group is 0, midlle channel is 5.0
results is [5, 20, 8, 26, 14, 3, 0, 19]
The electrode with the largest average ripple power is 2
The max channel is 8
The electrode with the largest average ripple power is 2
The max channel is 8
The electrode with the largest average ripple power is 2
The max channel is 8
group is 1, midlle channel is 24.0
results is [24, 6, 21, 16, 10, 17, 30, 2]
The electrode with the largest average ripple power is 4
The max channel is 10
The electrode with the largest average ripple power is 4
The max channel is 10
The electrode with the largest average ripple power is 6
The max channel is 30
group is 2, midlle channel is 27.0
results is [27, 18, 31, 12, 23, 28, 15, 11]
The electrode with the largest average ripple power is 7
The max channel is 11
The electrode with the largest average ripple power is 7
The max channel is 11
The electrode with the largest average ripple power is 7
The max channel is 11
group is 3, midlle channel is 50.0
results is [50, 40, 33, 47,

C:\Users\sachur\AppData\Local\Temp\ipykernel_4180\1053226298.py:155: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_file['ripple_ch'].iloc[i] = ripple_channel[unit_group.values[0]]
C:\Users\sachur\AppData\Local\Temp\ipykernel_4180\10532262

In [8]:
#### Test function for getting

import sys
sys.path.append(r'Q:\sachuriga\Sachuriga_Python\quattrocolo-nwb4fp\src/')
sys.path.append(r'Q:\sachuriga\Sachuriga_Python\cnn-ripple-main/src/')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nwb4fp.analyses.data import get_pkl_files, group_channels_by_group, find_middle_channel_per_group, map_middle_channels_to_input,get_nearest_8_by_position, load_speed_fromNWB, pos2speed, find_run_indices
from nwb4fp.preprocess.down_sample_lfp import down_sample_lfp_test
import pynapple as nap
from cnn.load_data import z_score_normalization, downsample_data
from cnn.utils import smooth2ripple
from cnn.load_data import generate_overlapping_windows


file_path = r"Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp/ASSY-236-F.prb"
base_folder = r"S:/Sachuriga/Ephys_Recording/CR_CA1"
base_nwb_fodler  =r"S:\Sachuriga/nwb/test4neo"


# Read the file and parse the dictionary
local_vars = {'np': np}
with open(file_path, 'r') as file:
    exec(file.read(), local_vars)  # Execute the file content with NumPy in scope

channel_groups = local_vars.get('channel_groups')
if channel_groups is None:
    raise ValueError(f"'channel_groups' not found in {file_path}")

# Assuming channel_groups is loaded from Step 1
data = []
for group_id, group_data in channel_groups.items():
    channels = group_data['channels']
    geometry = group_data['geometry']
    for channel in channels:
        x, y = geometry[channel]
        data.append({
            'group_id': group_id,
            'channel_id': channel,
            'x': x,
            'y': y
        })
probe_df = pd.DataFrame(data)
pkl_files = get_pkl_files(r"Q:/crhip\Sachuriga/filre_with_table/adjust_y_with _meanVAlue/")
base_tsne_folder = r"Q:/crhip\Sachuriga/filre_with_table/adjust_y_with _meanVAlue/"
base_tsne_ripplemax_folder = r"Q:/crhip\Sachuriga/filre_with_table/adjust_y_with _meanVAlue/clusters_with_tsneLabel/ripple_max"
all_dataframes = []

import pandas as pd
file = r"65165_2023-06-29_A_units_table_withDLC.pkl"
# Read and process the pickle file
print(fr"working on this folder {file}")
df_file = pd.read_pickle(fr"{base_tsne_folder}{file}")

#df_file = pd.read_pickle(fr"S:/Sachuriga/filre_with_table/adjust_y_with _meanVAlue/63385_2024-07-22_B_units_table_withDLC.pkl")
unit_table = pd.DataFrame(df_file)
pyramidal_df = unit_table[unit_table['cell_type']=="pyramidal"]

file_path_nwb = pyramidal_df['session_id'].iloc[0]
channel_list = pyramidal_df['ch'].values

# Channel grouping and calculations
grouped_channels = group_channels_by_group(channel_list)
middle_channels = find_middle_channel_per_group(grouped_channels)
output_list = map_middle_channels_to_input(channel_list, middle_channels)
animal_id = unit_table['matlab_animal'].iloc[0]
nwb_file = unit_table['session_id'].iloc[0]
phy_folder = unit_table['session_id'].iloc[0].split(".nwb")[0]
os_fodler = unit_table['session_id'].iloc[0].split("_phy_k_manual.nwb")[0]
temp_folder = r"C:/temp_lfp"
path = fr"{base_folder }/{animal_id}/{phy_folder}/"
raw_path = fr"{base_folder}/{animal_id}/{os_fodler}"
npdata = nap.load_file(fr"{base_nwb_fodler}/{nwb_file}")
#down_sample_lfp_test(path,raw_path)
pos_cord = load_speed_fromNWB(npdata['XY_mid_brain'])
## filter speed
raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                            pos_cord[:,1], # x
                            pos_cord[:,2], # y
                            filter_speed=True, 
                            min_speed = 0.1)
time_stemp = pos_cord[:,0]
starts,stops = find_run_indices(smoothed_speed, threshold=0.05)
run_ep = nap.IntervalSet(start=time_stemp[starts], end=time_stemp[stops], time_units='s')
wake_ep = nap.IntervalSet(start=time_stemp[0], end=time_stemp[-1], time_units='s')
rest_ep = wake_ep.set_diff(run_ep)
## find max
lfp = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_zscore.npy")
lfp_times = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/lfp_times.npy")
try:
    bad_channels = np.load(rf"{base_folder}/{animal_id}/{phy_folder}/bad_channels.npy")
except Exception as e:
    bad_channels = []
result_dict={}
ripple_channel={}
ripple_channel_3std={}
ripple_channel_4std={}
for group, ch in middle_channels.items():
    temp = probe_df[probe_df['group_id']==group]
    temp_chs = temp.sort_values('y', ascending=False)
    chs = temp_chs['channel_id'].values
    # Example usage
    numbers = list(chs)
    target = np.int64(ch)
    bad_channels=list(bad_channels)
    result = get_nearest_8_by_position(numbers, target, bad_channels)
    
    print(fr"group is {group}, midlle channel is {ch}")
    print(fr"results is {result}")

    # Data preprocessing (unchanged)
    #temp = [31, 12, 23, 28, 15, 29, 7, 13]  # Electrode indices
    temp_lfps = []
    temp_lfps_times = []
    temp = result
    # Extract LFP signals for selected electrodes
    for ch in temp:
        temp_lfp = lfp[:, ch]
        eeg_examples = nap.Tsd(t=lfp_times, d=temp_lfp, fs=1250)
        signal = eeg_examples.restrict(rest_ep).values
        t = eeg_examples.restrict(rest_ep).index.values
        temp_lfps.append(list(signal))
        temp_lfps_times = list(t)

    # Convert to numpy array and normalize
    raw_lfp = np.array(temp_lfps).T
    data = z_score_normalization(raw_lfp)
    swr_events, max_electrode = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=2)
    swr_events1, max_electrode1 = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=3)
    swr_events2, max_electrode2 = process_swr_analysis(data, temp, electrode_idx=result.index(ch), sampling_rate=1250, n_electrodes=8, amplitude_threshold=4)
    ripple_channel[group] = temp[max_electrode]
    ripple_channel_3std[group] = temp[max_electrode1]
    ripple_channel_4std[group] = temp[max_electrode2]
df_file['ripple_ch']=None
df_file['ripple_ch_3std']=None
df_file['ripple_ch_4std']=None
df_file['shank_group']=None
error_log=[]
for i in range(len(df_file)):
    unit_group = probe_df[probe_df['channel_id']==df_file['ch'].iloc[i]]['group_id']
    try:
        df_file['ripple_ch'].iloc[i] = ripple_channel[unit_group.values[0]]
        df_file['ripple_ch_3std'].iloc[i] = ripple_channel_3std[unit_group.values[0]]
        df_file['ripple_ch_4std'].iloc[i] = ripple_channel_4std[unit_group.values[0]]
        df_file['shank_group'].iloc[i] = unit_group.values[0]
    except Exception as e:
        error_log.append(e)
        continue
df_file.to_pickle(fr"{base_tsne_ripplemax_folder}/{file}")

working on this folder 65165_2023-06-29_A_units_table_withDLC.pkl


IndexError: single positional indexer is out-of-bounds

In [1]:
import numpy as np
from scipy.signal import hilbert, butter, filtfilt, find_peaks
from scipy.ndimage import gaussian_filter1d

# # Data preprocessing (unchanged)
# #temp = [31, 12, 23, 28, 15, 29, 7, 13]  # Electrode indices
# temp_lfps = []
# temp_lfps_times = []

# # Extract LFP signals for selected electrodes
# for ch in temp:
#     temp_lfp = lfp[:, ch]
#     eeg_examples = nap.Tsd(t=lfp_times, d=temp_lfp, fs=1250)
#     signal = eeg_examples.restrict(rest_ep).values
#     t = eeg_examples.restrict(rest_ep).index.values
#     temp_lfps.append(list(signal))
#     temp_lfps_times = list(t)

# # Convert to numpy array and normalize
# raw_lfp = np.array(temp_lfps).T
# data = z_score_normalization(raw_lfp)

# Existing functions (unchanged)
def detect_swrs(
    lfp_data,
    electrode_idx,
    sampling_rate,
    ripple_band=(70, 250),
    amplitude_threshold=4,
    min_duration=0.05,
    max_duration=2.0,
    gaussian_std_ms=12.5
):
    """
    Detect sharp wave ripples (SWRs) in LFP data from a single electrode.
    [Docstring unchanged]
    """
    # Extract LFP for the chosen electrode
    lfp = lfp_data[:, electrode_idx]

    # Apply bandpass filter
    nyquist = sampling_rate / 2
    lowcut, highcut = ripple_band
    b, a = butter(4, [lowcut / nyquist, highcut / nyquist], btype='band')
    filtered_lfp = filtfilt(b, a, lfp)

    # Compute amplitude envelope
    amplitude_envelope = np.abs(hilbert(filtered_lfp))

    # Smooth envelope with Gaussian kernel
    gaussian_std_samples = gaussian_std_ms * sampling_rate / 1000
    smoothed_envelope = gaussian_filter1d(amplitude_envelope, sigma=gaussian_std_samples)

    # Calculate detection threshold
    mean_envelope = np.mean(smoothed_envelope)
    std_envelope = np.std(smoothed_envelope)
    threshold = mean_envelope + amplitude_threshold * std_envelope

    # Find peaks above threshold
    peak_indices = find_peaks(smoothed_envelope, height=threshold)[0]

    # Detect SWR events
    swr_events = []
    min_samples = int(min_duration * sampling_rate)
    max_samples = int(max_duration * sampling_rate)

    for peak_idx in peak_indices:
        # Find start of event
        start_idx = peak_idx
        while start_idx > 0 and smoothed_envelope[start_idx] > mean_envelope:
            start_idx -= 1
        if start_idx > 0:
            start_idx += 1

        # Find end of event
        end_idx = peak_idx
        while end_idx < len(smoothed_envelope) - 1 and smoothed_envelope[end_idx] > mean_envelope:
            end_idx += 1
        if end_idx < len(smoothed_envelope) - 1:
            end_idx -= 1

        # Validate event duration
        if min_samples <= (end_idx - start_idx) <= max_samples:
            swr_events.append((start_idx, end_idx, peak_idx))

    return swr_events

def butter_bandpass(lowcut, highcut, fs, order=4):
    """Design a bandpass Butterworth filter."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """Apply a bandpass filter to the data."""
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data, axis=0)

def find_max_electrode(data, fs, n_electrodes, events):
    """
    Identify the electrode with the largest average ripple power.
    [Docstring unchanged]
    """
    # Filter signal in ripple band
    filtered_data = bandpass_filter(data, 100, 250, fs)
    squared_data = filtered_data ** 2

    # Compute average power across events
    average_powers = np.zeros(n_electrodes)
    times_rip = np.zeros(n_electrodes)
    for start, end, peak in events:
        segment = squared_data[start:peak, :]
        power = np.mean(segment, axis=0)
        average_powers += power
        times_rip += (peak-start)
    average_powers /= len(events)
    #average_powers_sum /= times_rip 
    # Identify electrode with maximum power
    max_electrode = np.argmax(average_powers)
    print(f"The electrode with the largest average ripple power is {max_electrode}")
    return max_electrode

# New function wrapping the main execution
def process_swr_analysis(data, temp, electrode_idx=0, sampling_rate=1250, n_electrodes=8, amplitude_threshold=2):
    """
    Process LFP data to detect SWRs and find the electrode with maximum ripple power.

    Parameters
    ----------
    data : ndarray
        Normalized LFP data.
    temp : list
        List of electrode indices.
    electrode_idx : int, optional
        Index of the initial electrode for SWR detection (default: 0).
    sampling_rate : float, optional
        Sampling rate in Hz (default: 1250).
    n_electrodes : int, optional
        Number of electrodes (default: 8).

    Returns
    -------
    swr_events : list
        List of detected SWR events.
    max_electrode : int
        Index of the electrode with maximum ripple power.
    """
    lfp_data = data
    fs = sampling_rate

    # Detect SWRs and find max electrode
    swr_events = detect_swrs(lfp_data, electrode_idx, sampling_rate, amplitude_threshold=amplitude_threshold)
    max_electrode = find_max_electrode(data, fs, n_electrodes, swr_events)
    print(f"The max channel is {temp[max_electrode]}")
    
    return swr_events, max_electrode

# Example call to the new function
# swr_events, max_electrode = process_swr_analysis(data, temp)